## Usage

In [4]:
from python_middlewareable import MiddlewarePipeline

pipeline = (
    (MiddlewarePipeline[int]())
    .pipe(lambda x, next_call: next_call(x + 1))
    .pipe(lambda x, next_call: next_call(x * 2))
    .pipe(lambda x, next_call: next_call(x + 3))
)

# output: 11
print(await pipeline.process(3))

# output: 15
print(await pipeline.process(5))

11
15


In [5]:
##
# Using the same processor for multiple pipelines
##
from python_middlewareable import MiddlewarePipeline, MiddlewareProcessor, MiddlewareCallable

# Initialize the processor
processor = MiddlewareProcessor()

# Define stages with type hints
stage1: MiddlewareCallable[int, int] = lambda x, next_call: next_call(x + 1)
stage2: MiddlewareCallable[int, int] = lambda x, next_call: next_call(x * 2)
stage3: MiddlewareCallable[int, int] = lambda x, next_call: next_call(x + 3)

# Create pipelines that use the same processor
pipeline1 = MiddlewarePipeline(processor=processor).pipe(stage1).pipe(stage2)
pipeline2 = MiddlewarePipeline(processor=processor).pipe(stage3)

# Process payloads
result1 = await pipeline1.process(5)
result2 = await pipeline2.process(5)

# output: 12
print(result1)

# output: 8
print(result2)

12
8


In [6]:
# import classes
from dataclasses import dataclass
from python_middlewareable import (
    MiddlewareStage,
    MiddlewareNextCall,
    MiddlewarePipeline,
)


# define a payload
@dataclass
class Request:
    value: str


# create a middleware
class OneMiddleware(MiddlewareStage[Request]):
    async def handle(
        self,
        request: Request,
        next_call: MiddlewareNextCall,
    ) -> Request:
        # do something before the next middleware
        print("OneMiddleware before")

        # call the next middleware
        result = await next_call(request)

        # do something after the next middleware
        print("OneMiddleware after")
        request.value = request.value + " from OneMiddleware"

        return result


# add the `middlewarePipeline` mixin to the class that will use the middleware
class App(MiddlewarePipeline[Request]):
    stages = [OneMiddleware]


# instantiate and use it
app = App()
request = Request(value="Hello")
response = await app.process(request)

# check the result
print(response)

# output:
# OneMiddleware before
# OneMiddleware after
# Request(value='Hello from OneMiddleware')

OneMiddleware before
OneMiddleware after
Request(value='Hello from OneMiddleware')
